In [37]:
import pandas as pd
import networkx as nx
import csv
from xml.dom import minidom
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import tostring



#-------- Variables --------------------------------------------------------------------

list_filename = []
list_name = []
list_accession = []
list_scientific_name = []
list_genes_total = []
list_genes_protein_coding = []
list_mrna = []
list_cds = []
list_model_refseq = []
list_known_refseq = []
list_cds_fully_supported = []
list_cds_ab_initio = []
list_length_count_genes = []
list_length_mean_genes = []
list_length_count_transcripts = []
list_length_mean_transcripts = []
list_length_count_cds = []
list_length_mean_cds = []
list_length_count_introns = []
list_length_mean_introns = []
list_length_count_exons = []
list_length_mean_exons = []
list_proteins_seq = []
list_transcripts_seq = []
list_proteins_hsapiens_seq = []
list_transcripts_hsapiens_seq = []
list_rnaseq_reads = []
list_num_tissues = []
list_num_runs = []


#-------- Read xml files ---------------------------------------------------------------
list_filenames = pd.read_csv('File_names.csv', header=0, sep='\t', low_memory=False)
it=list(range(0, len(list_filenames.xml)))


    
for i in it:
    #1
    filename=list_filenames.xml.iloc[i]
    list_filename.append(filename)

    tree= ET.parse(filename)
    root = tree.getroot()
    
    #2
    m=root.findall("./AssembliesReport/FullAssembly/Name")
    name=m[0].text
    list_name.append(name)

    #3
    m=root.findall("./AssembliesReport/FullAssembly/Accession")
    accession=m[0].text
    list_accession.append(accession)

    #4
    m=root.findall(".//ScientificName")
    if not m:
        m='Error'
    else:
        m=m[0].text
    scientific_name=m
    list_scientific_name.append(scientific_name)
    
    #5
    m = root.find(".//Counts[@feature_type='genes']/Total")
    genes_total=m.text
    list_genes_total.append(genes_total)
    
    #6
    m = root.find(".//Counts[@feature_type='genes']/Subcount[@subtype='protein coding']")
    genes_protein_coding=m.text
    list_genes_protein_coding.append(genes_protein_coding)

    #7
    m = root.find(".//Counts[@feature_type='mRNAs']/Total")
    mrna=m.text
    list_mrna.append(mrna)
    
    #8
    m = root.find(".//Counts[@feature_type='CDSs']/Total")
    cds=m.text
    list_cds.append(cds)
    
    #9
    m = root.find(".//Counts[@feature_type='CDSs']/Subcount[@subtype='model RefSeq']")
    model_refseq=m.text
    list_model_refseq.append(model_refseq)
    
    
    #10
    m = root.find(".//Counts[@feature_type='CDSs']/Subcount[@subtype='known RefSeq']")
    known_refseq=m.text
    list_known_refseq.append(known_refseq)
    
    #11
    m = root.find(".//Counts[@feature_type='CDSs']/Subcount[@subtype='fully supported']")
    cds_fully_supported=m.text
    list_cds_fully_supported.append(cds_fully_supported)
    
    
    #12
    m = root.find(".//Counts[@feature_type='CDSs']/Subcount[@subtype='ab initio > 5%']")
    cds_ab_initio=m.text
    list_cds_ab_initio.append(cds_ab_initio)
    
    #13
    m = root.find(".//LengthStats[@category='Genes']/Count")
    length_count_genes=m.text
    list_length_count_genes.append(length_count_genes)
    
    #14
    m = root.find(".//LengthStats[@category='Genes']/Mean")
    length_mean_genes=m.text
    list_length_mean_genes.append(length_mean_genes)
    
    #15
    m = root.find(".//LengthStats[@category='All transcripts']/Count")
    list_length_count_transcripts.append(m.text)
        
    #16
    m = root.find(".//LengthStats[@category='All transcripts']/Mean")
    list_length_mean_transcripts.append(m.text)
    
    #17
    m = root.find(".//LengthStats[@category='CDSs']/Count")
    list_length_count_cds.append(m.text)
    
    #18
    m = root.find(".//LengthStats[@category='CDSs']/Mean")
    list_length_mean_cds.append(m.text)
    
    #19
    m = root.find(".//LengthStats[@category='All exons']/Count")
    list_length_count_exons.append(m.text)
    
    #20
    m = root.find(".//LengthStats[@category='All exons']/Mean")
    list_length_mean_exons.append(m.text)
    
    #21
    m = root.find(".//LengthStats[@category='All introns']/Count")
    list_length_count_introns.append(m.text)
    
    #22
    m = root.find(".//LengthStats[@category='All introns']/Mean")
    list_length_mean_introns.append(m.text)

    
    #Find name Accession of Primary Assembly
    mx=root.findall(".//AssemblyUnit/Name")  
    n="Primary Assembly"
    it=list(range(0, len(mx)))
    for i in it:
        if mx[i].text==n:
            #print(mx[i].text)
            nm=root.findall(".//AssemblyUnit/Accession") 
            name=nm[i].text

            
    #Number of proteins from Primary Assembly
    #mx = root.find(".//ProteinAlignStats")  
    mx=root.find(f'.//ProteinAlignStats/AssemblyStats[@accession="{name}"]')  
    if mx is not None:
        #print("Variable is not None")
        m = mx.findall(".//TotalSequences")
        it=list(range(0, len(m)))
        x=[m[i].text for i in it]
        m=[int(k) for k in x]
        num_seq=sum(m)
    else:
        #print("Variable is None")
        num_seq=0
    list_proteins_seq.append(num_seq)
    
    
    #Number of proteins from Primary Assembly with "Species = Homo Sapiens"
    mx=root.find(f'.//ProteinAlignStats/AssemblyStats[@accession="{name}"]')  
    if mx is not None:
        #print("Variable is not None")
        m = mx.findall(".//CategoryStats[@Species='Homo sapiens']/TotalSequences")
        #print(m[0].text)
        it=list(range(0, len(m)))
        x=[m[i].text for i in it]
        m=[int(k) for k in x]
        num_seq=sum(m)
    else:
        #print("Variable is None")
        num_seq=0
    list_proteins_hsapiens_seq.append(num_seq)

    
    
    #Number of transcripts from Primary Assembly
    mx=root.find(f'.//TranscriptAlignStats/AssemblyStats[@accession="{name}"]')     
    if mx is not None:
        #print("Variable is not None")
        m=mx.findall(".//TotalSequences")
        it=list(range(0, len(m)))
        x=[m[i].text for i in it]
        m=[int(k) for k in x]
        num_seq=sum(m)
    else:
        #print("Variable is None")
        num_seq=0
    list_transcripts_seq.append(num_seq)
    

    #Number of transcripts from Primary Assembly with "Species = Homo Sapiens"
    mx=root.find(f'.//TranscriptAlignStats/AssemblyStats[@accession="{name}"]')     
    if mx is not None:
        #print("Variable is not None")
        m = mx.findall(".//CategoryStats[@Species='Homo sapiens']/TotalSequences")
        #print(m[0].text)
        it=list(range(0, len(m)))
        x=[m[i].text for i in it]
        m=[int(k) for k in x]
        num_seq=sum(m)
    else:
        #print("Variable is None")
        num_seq=0
    list_transcripts_hsapiens_seq.append(num_seq)
    
    
    #Num RNA-Seq reads
    mx=root.find(f'.//RnaseqAlignReport/AssemblyUnit[@accession="{name}"]')  
    if mx is not None:
        #print("Variable is not None")
        m = mx.findall(".//SampleStats[@sample='multiple']/NumReads")
        num_seq=m[0].text
    else:
        num_seq=0
    list_rnaseq_reads.append(num_seq)
    
    
    #Number of tissues
    list_tissues = []
    mx=root.find(f'.//RnaseqAlignReport/AssemblyUnit[@accession="{name}"]')  
    if mx is not None:
        #print("Variable is not None")
        m = mx.findall(".//SampleName")
        itx=list(range(0, len(m)))
        for j in itx:
            tissue=m[j].text
            pos=tissue.find('(')
            start=0
            substring = tissue[start:pos]
            list_tissues.append(substring)
        list_tissues = list(set(list_tissues)) 
        list_tissues.remove("Aggregate of all aligned sample")
        #print(list_tissues)
        num_tissues=len(list_tissues)
    else:
        num_tissues=0      
    list_num_tissues.append(num_tissues)
    
    
    #Number of runs
    mx=root.find(f'.//RnaseqAlignReport/AssemblyUnit[@accession="{name}"]')  
    if mx is not None:
        #print("Variable is not None")
        m = mx.findall(".//RunStats")
        num_runs=len(m)
    else:
        num_runs=0 
    list_num_runs.append(num_runs)




            
#-------- Output Data ---------------------------------------------------------------
data = pd.DataFrame()
data = pd.DataFrame(data,columns=['FileName', 'Name','Accession','ScientificName','Genes_total','Genes_protein_coding','mRNAs','CDSs','CDSs_model_refseq','CDSs_known_refseq','CDSs_fully_supported','CDSs_ab_initio','Length_count_genes','Length_mean_genes','Length_count_transcripts','Length_mean_transcripts','Length_count_cds','Length_mean_cds','Length_count_exons','Length_mean_exons','Length_count_introns','Length_mean_introns',
'Num_proteins','Num_transcripts','Num_proteins_hsapiens','Num_transcripts_hsapiens','Num_rnaseq_reads','Num_tissues','Num_runs'])


    
data.FileName=list_filename
data.Name=list_name
data.Accession=list_accession
data.ScientificName=list_scientific_name
data.Genes_total=list_genes_total
data.Genes_protein_coding=list_genes_protein_coding
data.mRNAs=list_mrna
data.CDSs=list_cds
data.CDSs_model_refseq=list_model_refseq
data.CDSs_known_refseq=list_known_refseq
data.CDSs_fully_supported=list_cds_fully_supported
data.CDSs_ab_initio=list_cds_ab_initio
data.Length_count_genes=list_length_count_genes
data.Length_mean_genes=list_length_mean_genes
data.Length_count_transcripts=list_length_count_transcripts
data.Length_mean_transcripts=list_length_mean_transcripts
data.Length_count_cds=list_length_count_cds
data.Length_mean_cds=list_length_mean_cds
data.Length_count_introns=list_length_count_introns
data.Length_mean_introns=list_length_mean_introns
data.Length_count_exons=list_length_count_exons
data.Length_mean_exons=list_length_mean_exons  
data.Num_proteins_hsapiens=list_proteins_hsapiens_seq
data.Num_transcripts_hsapiens=list_transcripts_hsapiens_seq   
data.Num_proteins=list_proteins_seq
data.Num_transcripts=list_transcripts_seq
data.Num_rnaseq_reads=list_rnaseq_reads
data.Num_tissues=list_num_tissues
data.Num_runs=list_num_runs



    
    
data.to_csv('Metadata.csv', sep='\t')
#data
    




    
#-------- Read Assembly txt files --------------------------------------------------

list_filename_assembly = []
list_assembly_name = []
list_refseq_assembly_accession = []
list_spanned_gaps = []
list_unspanned_gaps = []
list_scaffold_count = []
list_scaffold_N50 = []
list_contig_count = []
list_contig_N50 = []
list_total_gap_length = []
list_gc_perc = []
            
  
list_filenames = pd.read_csv('/home/rebeca/Escritorio/Data/Assembly/File_names.csv', header=0, sep='\t', low_memory=False)
it=list(range(0, len(list_filenames)))

for i in it:
    
    filename=list_filenames.iloc[i].filename    
    list_filename_assembly.append(filename)

    f=f"/home/rebeca/Escritorio/Data/Assembly/{filename}"
    file = open(f,'r')
    
    split_contig_count=0
    split_contig_N50=0
    split_total_gap_length=0
    split_gc_perc=0
    while True:
        content=file.readline()
        pos=content.find('# Assembly name:')
        if pos==0:
            split=content.split("# Assembly name:",1)[1]
            split = split.replace(" ", "")
            list_assembly_name.append(split)
        
        pos=content.find('# RefSeq assembly accession:')
        if pos==0:
            split=content.split("# RefSeq assembly accession:",1)[1]
            split = split.replace(" ", "")
            list_refseq_assembly_accession.append(split)
             
        pos=content.find('all	all	all	all	spanned-gaps')
        if pos==0:
            split=content.split("all	all	all	all	spanned-gaps",1)[1]
            split = split.replace("	", "")
            list_spanned_gaps.append(split)

        
        pos=content.find('all	all	all	all	unspanned-gaps')
        if pos==0:
            split=content.split("all	all	all	all	unspanned-gaps",1)[1]
            split = split.replace("	", "")
            list_unspanned_gaps.append(split)

        
        pos=content.find('all	all	all	all	scaffold-count')
        if pos==0:
            split=content.split("all	all	all	all	scaffold-count",1)[1]
            split = split.replace("	", "")
            list_scaffold_count.append(split)

        
        pos=content.find('all	all	all	all	scaffold-N50')
        if pos==0:
            split=content.split("all	all	all	all	scaffold-N50",1)[1]
            split = split.replace("	", "")
            list_scaffold_N50.append(split)

        
        pos=content.find('all	all	all	all	contig-count')
        if pos==0:
            split=content.split("all	all	all	all	contig-count",1)[1]
            split = split.replace("	", "")
            split_contig_count=split
        
        pos=content.find('all	all	all	all	contig-N50')
        if pos==0:
            split=content.split("all	all	all	all	contig-N50",1)[1]
            split = split.replace("	", "")
            split_contig_N50=split
            
        
        pos=content.find('all	all	all	all	total-gap-length')
        if pos==0:
            split=content.split("all	all	all	all	total-gap-length",1)[1]
            split = split.replace("	", "")
            split_total_gap_length=split

        
        pos=content.find('all	all	all	all	gc-perc')
        if pos==0:
            split=content.split("all	all	all	all	gc-perc",1)[1]
            split = split.replace("	", "")
            split_gc_perc=split
            
        if not content:
            break

    file.close()
    list_contig_count.append(split_contig_count)
    list_contig_N50.append(split_contig_N50)
    list_total_gap_length.append(split_total_gap_length)
    list_gc_perc.append(split_gc_perc)
        
    
data_assembly = pd.DataFrame()
data_assembly = pd.DataFrame(data_assembly,columns=['filename_assembly','assembly_name','refseq_assembly_accession','spanned_gaps','unspanned_gaps','scaffold_count','scaffold_N50','contig_count','contig_N50','total_gap_length','gc_perc'
])

data_assembly.filename_assembly=list_filename_assembly
data_assembly.assembly_name=list_assembly_name
data_assembly.refseq_assembly_accession=list_refseq_assembly_accession
data_assembly.spanned_gaps=list_spanned_gaps
data_assembly.unspanned_gaps=list_unspanned_gaps
data_assembly.scaffold_count=list_scaffold_count
data_assembly.scaffold_N50=list_scaffold_N50
data_assembly.contig_count=list_contig_count
data_assembly.contig_N50=list_contig_N50
data_assembly.total_gap_length=list_total_gap_length
data_assembly.gc_perc=list_gc_perc

data_assembly.to_csv('Metadata_Assembly.csv', sep='\t')
data_assembly

,filename_assembly,assembly_name,refseq_assembly_accession,spanned_gaps,unspanned_gaps,scaffold_count,scaffold_N50,contig_count,contig_N50,total_gap_length,gc_perc
0,GCF_000001635.27_GRCm39_assembly_stats.txt,GRCm39\n,GCF_000001635.27\n,204\n,143\n,102\n,106145001\n,306\n,59462871\n,73600614\n,42\n
1,GCF_000002035.6_GRCz11_assembly_stats.txt,GRCz11\n,GCF_000002035.6\n,17835\n,925\n,2848\n,4737936\n,20680\n,854399\n,4691662\n,36.5\n
2,GCF_000002425.4_Phypa_V3_assembly_stats.txt,PhypaV3\n,GCF_000002425.4\n,2643\n,0\n,359\n,17435539\n,3002\n,465069\n,5980520\n,33.5\n
3,GCF_000002775.5_P.trichocarpa_v4.1_assembly_st...,P.trichocarpa_v4.1\n,GCF_000002775.5\n,0\n,59\n,106\n,13159747\n,106\n,13159747\n,590000\n,34\n
4,GCF_000003025.6_Sscrofa11.1_assembly_stats.txt,Sscrofa11.1\n,GCF_000003025.6\n,413\n,93\n,706\n,88231837\n,1118\n,48231277\n,29864641\n,42\n
...,...,...,...,...,...,...,...,...,...,...,...
688,GCF_963924715.1_fThuThy2.1_assembly_stats.txt,fThuThy2.1\n,GCF_963924715.1\n,468\n,0\n,119\n,34318521\n,587\n,2956553\n,93600\n,40\n
689,GCF_963930625.1_mMunRee1.1_assembly_stats.txt,mMunRee1.1\n,GCF_963930625.1\n,682\n,0\n,267\n,114544513\n,949\n,5780196\n,136400\n,42\n
690,GCF_963930695.1_fLabBer1.1_assembly_stats.txt,fLabBer1.1\n,GCF_963930695.1\n,815\n,0\n,131\n,31261685\n,946\n,2715806\n,163000\n,40.5\n
691,GCF_963932015.1_bAnaAcu1.1_assembly_stats.txt,bAnaAcu1.1\n,GCF_963932015.1\n,631\n,0\n,394\n,76435830\n,1025\n,3416933\n,125881\n,42.5\n


it=list(range(0, len(list_filenames.xml)))

for i in it:
    filename=list_filenames.xml.iloc[i]
    print(filename)

### 